In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
spark_context = SparkContext("local", "first_app")
spark = SparkSession(spark_context)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sql_context = SQLContext(spark_context)

In [2]:
text_file = sql_context.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', quote='"', delimiter=',').load("/home/jovyan/work/Covid19.csv")
rdd_filter = text_file.rdd.map(tuple)
rdd_covid = rdd_filter.map(lambda word: (word[7], (word[5], word[6], word[2])))
rdd_covid = rdd_covid.filter(lambda word: word[1][2] == 8)
rdd_covid = rdd_covid.map(lambda word: (word[0], (word[1][0], word[1][1])))
rdd_covid.take(5)

[('Afghanistan', (19, 0)),
 ('Afghanistan', (3, 0)),
 ('Afghanistan', (11, 1)),
 ('Afghanistan', (3, 0)),
 ('Afghanistan', (55, 4))]

In [3]:
rdd_covid = rdd_covid.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]))
print(rdd_covid.take(5))

[('Afghanistan', (1620, 131)), ('Argentina', (222243, 5090)), ('Armenia', (5200, 139)), ('Aruba', (1877, 7)), ('Australia', (9367, 422))]


In [4]:
rdd_covid = rdd_covid.filter(lambda word: word[0] in ["Guatemala"])
print(rdd_covid.collect())

[('Guatemala', (25086, 873))]


In [5]:
covid_cases = rdd_covid.map(lambda x: x[1][0])
covid_deaths = rdd_covid.map(lambda x: x[1][1])
covid_data = [covid_cases.collect()[0], covid_deaths.collect()[0]]
covid_data

[25086, 873]

In [6]:
fig = go.Figure(data=go.Bar(x=["Cases", "Deaths"],y=covid_data))
fig.update_layout(title_text='Covid-19 cases and deaths in Guatemala in august',title_font_size=30,
                  yaxis=dict(title='Total',title_font_size=25),
                  xaxis=dict(title='Type',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('report7.html', auto_open=True)